<a href="https://colab.research.google.com/github/akashe/arxiv_hunter/blob/main/Summarization_using_BART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Idea: Try out simple summarization using BART model. 
Why BART:
1. Really good ROUGE-1 score. A significant jump in Rouge-1 scores. Later models do improve the score but aren't a really big jump.

Things from BART paper(https://arxiv.org/pdf/1910.13461v1.pdf):
1. You could use the entire seq-2-seq model as pretrained model while fine-tuning. For example, for translation tasks they use the pretrained english to english model and for encoder they put another transformer to convert initial foreign language to masked english tokens.
2. Maksing: It seems from multiple papers that masking does help in better language understanding(tasks like SQuad). However, as they mention in the paper, it isn't good for language generation tasks. 
3. BART seems to work better for language generation tasks beacuse of the cross-attention between source and target sequences.

In [1]:
!pip install transformers

     |████████████████████████████████| 2.3MB 27.2MB/s 
     |████████████████████████████████| 901kB 45.5MB/s 
     |████████████████████████████████| 3.3MB 42.3MB/s 


In [2]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

In [3]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

In [4]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

In [18]:
ARTICLE_TO_SUMMARIZE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

In [19]:
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=130, return_tensors='pt')

In [20]:
summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=5, early_stopping=True)

In [21]:
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

['NewNew York']
